In [ ]:
require 'gsl'

module Rng
  def rng(seed=0)
    @rng ||= GSL::Rng.alloc 'gsl_rng_mt19937', seed
  end
end

class AdPublisher
  include Rng
  
  def initialize(scale, min_price, relative_width=0.1)
    @scale = scale
    @min = min_price
    @width = relative_width
  end
  
  def work(price)
    return 0 if price <= @min
    
    mean = @scale * Math.log(price / @min)
    demand = mean + rng.gaussian(@width * mean)
    
    [0, demand].max
  end
end

In [ ]:
publisher = AdPublisher.new 100, 2
result = []
1.upto(100).each do |i|
  price = i/100.0 * 20
  10.times do
    result << [price, publisher.work(price)]
  end
end

require 'gnuplot'

Gnuplot::Plot.new do |plot|
  plot.xlabel 'Price per impression'
  plot.ylabel 'Impressions'
  plot.data << Gnuplot::DataSet.new(result.transpose) do |ds|
    ds.notitle
  end
  plot.data << Gnuplot::DataSet.new('20 * x')
end